In [1]:
import os
import json
import pandas as pd
import numpy as np
from scipy import stats
from sklearn import linear_model
import statsmodels.api as sm

import matplotlib.pyplot as plt
import seaborn as sns
import plotly 
import plotly.graph_objects as go
import plotly.express as px

%matplotlib qt
pd.set_option('mode.chained_assignment', None)

# Process modules

In [64]:
with pd.HDFStore(r'Experiments\MultiplyTemperature\Exp0(0)\Exp0_up.hdf5') as file:
        data = file['data']
        print( file.get_storer('data').attrs.__dict__)

{'_v__nodefile': File(filename=Experiments\MultiplyTemperature\Exp0(0)\Exp0_up.hdf5, title='', mode='a', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/data (Group) ''
/data/axis0 (Array(6,)) ''
  atom := StringAtom(itemsize=19, shape=(), dflt=b'')
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := None
/data/axis1 (Array(17827,)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None
/data/block0_items (Array(3,)) ''
  atom := StringAtom(itemsize=4, shape=(), dflt=b'')
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := None
/data/block0_values (Array(17827, 3)) ''
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None
/data/block1_items (Array(2,)) ''
  atom := StringAtom(itemsize=19, shape=(), dflt=b'')


In [87]:
import copy

In [92]:
a

In [91]:
b =

In [108]:
class Container:

    data: pd.DataFrame = None
    

    def __init__(self, data=None, name=''):
        self.data = data
        self.folder=None
        self.experiment_name = name
        self._log = []
        self.info = {}

    def load_csv(self, path=''):
        path, self.folder, self.experiment_name = self._input_path(path)

        self.data = pd.read_csv(path)
        print(f'Loaded: {self.experiment_name}')
        self.log(f'csv loaded from: {path}')
        return self.data

    def load_hdf5(self, path=''):
        path, self.folder, self.experiment_name = self._input_path(path)

        with pd.HDFStore(path) as file:
            data = file['data']
            meta = file.get_storer('data').attrs.meta
        
        self._log.extend(meta['log'])
        self.log(f'hdf5 loaded from: {path}')
        meta.pop('log')
        self.info.update(meta)
        return data

    def dump(self,**kwargs):
        path = f'{self.folder}\{self.experiment_name}'
        kwargs.update({'log':self.log()})
        with pd.HDFStore(f'{path}.hdf5') as file:
            file.put('data', self.data)
            file.get_storer('data').attrs.meta = kwargs 
                

    def initial_filter(
            self,
            time_in_minutes=False,
            temperature=(-np.inf, np.inf),
            viscosity=(0, np.inf),
            time=(0, np.inf),
    ):
        conditions = dict(
            temperature=temperature,
            viscosity=viscosity,
            time=time,
        )

        temperature_cond = (
            (temperature[0] < self.data['Temperature']) &\
            (self.data['Temperature'] < temperature[1])
            )
        viscosity_cond = (
            (viscosity[0] < self.data['Viscosity']) &\
            (self.data['Viscosity'] < viscosity[1])
            )
        time_cond = (
            (time[0] < self.data['time']) &\
            (self.data['time'] < time[1])
            )

        self.data = self.data[
            temperature_cond &\
            viscosity_cond &\
            time_cond]
        if time_in_minutes: self.data['time'] / 60
        self.log(('initial_filter', conditions))
        return self.data

    def mask_filter(
        self,
        filter_func,
        by='Temperature',
        column='Viscosity',
    ):
        group = self.data.groupby(by=by)[column]
        mask = group.apply(filter_func).droplevel([0]).sort_index().to_numpy()
        self.data = self.data[mask]
        self.log(('mask_filter', filter_func.__name__))
        return self.data

    def linearize(self):
        T_func = lambda t: 1 / (t+273.15)
        v_func = lambda nu: np.log(nu)
        self.data['Temperature'] = self.data['Temperature'].apply(T_func)
        self.data['Viscosity'] = self.data['Viscosity'].apply(v_func)
        self.log('linearize')
        return self.data

    def copy(self):
        new =copy.deepcopy(self)
        return new

    def log(self, msg=None):
        if msg is not None: self._log.append(msg)
        return self._log

    @staticmethod
    def _input_path(path):
        while (path == '') or (not os.path.isfile(path)):
            path = input(f"Input data path: ")
        _path_list = (path).split('\\')
        folder = '\\'.join(_path_list[:-1])
        experiment_name = _path_list[-1].split('.')[0]
        return path, folder, experiment_name


In [3]:
## Filtration
def z_filter(data: pd.Series):
    mean = data.mean()
    s = data.std(ddof=0)+1e-50
    z_score= np.abs((data-mean) / s) <1
    return z_score

# def z_filter(data: pd.Series):
#     return stats.zscore(data)<1

def whisker_iqr_filter(data: pd.Series):
    whisker_width=0.5
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    return (data >= q1 - whisker_width*iqr) & (data <= q3 + whisker_width*iqr)

def iqr_filter(data: pd.Series):
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    iqr = q3 - q1
    
    return np.abs((data - data.median()) / iqr) < 1

def my_z_filter(data: pd.Series):
    mean = data.median()
    s = data.std()+1e-50
    z_score= np.abs((data-mean) / s) <0.1
    return z_score

# Load raw and save ready data

In [ ]:
def read_raw(path='',index_col=None):
    path = ''
    while (path == '') or (not os.path.isfile(path)):
        path = input(f"Input data path: ")
    _path_list = (path).split('\\')
    folder_path = '\\'.join(_path_list[:-1])
    data_name = _path_list[-1]
    print(f'    Folder:\n{folder_path}\n    Data name:\n {data_name}')
    return (folder_path,data_name), pd.read_csv(path,index_col=index_col)

In [ ]:
(folder_path,data_name),df = read_raw(index_col=0)
df

In [ ]:
rules = {
    1: 'image_sweep_check',
    2: 'combine_check',  # 'inner_processor_check':'OK_inner'
    0: 'OK'
}
col_rules = {
    'Viscosity_mark': 'Viscosity_verbose',
    'Temperature_mark': 'Temperature_verbose',
}

# df1 = df1.rename(columns=col_rules)
# df1[['Viscosity_verbose','Temperature_verbose']] =\
#     df1[['Viscosity_verbose','Temperature_verbose']].replace(rules)

# df2 = df2.rename(columns=col_rules)
# df2[['Viscosity_verbose','Temperature_verbose']] = df2[['Viscosity_verbose','Temperature_verbose']].replace(rules)
# df2['time'] =df2['time']+ df1['time'].max()+1

# df = pd.concat([df1,df2])
np.any(df['time'].duplicated())

In [ ]:
df.dropna().to_csv(f'{folder_path}\{data_name}', index=0)

# Load

In [109]:
a = Container()
a.load_csv('Experiments\MultiplyTemperature\Exp0(0)\Exp0_up.csv')

Loaded: Exp0_up


,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
0,1.0,344.9,OK,13.8,OK
1,2.0,344.9,OK,13.8,OK
2,3.0,344.9,OK,13.8,OK
3,4.0,345.4,image_sweep_check,13.8,OK
4,5.0,345.4,image_sweep_check,13.8,OK
...,...,...,...,...,...
18502,19333.0,45.7,OK,40.5,OK
18503,19334.0,46.0,OK,40.5,OK
18504,19335.0,45.6,OK,40.5,image_sweep_check
18505,19336.0,45.7,OK,40.5,OK


# Process

In [110]:
a.initial_filter(time=(63, np.inf), viscosity=(40, 350), temperature=(13, 40))

,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
63,64.0,343.9,image_sweep_check,13.8,OK
64,65.0,344.4,OK,13.8,OK
65,66.0,344.4,OK,13.8,OK
66,67.0,344.4,OK,13.8,OK
67,68.0,344.9,OK,13.8,OK
...,...,...,...,...,...
18282,19111.0,48.9,OK,39.9,OK
18283,19112.0,49.3,OK,39.9,OK
18284,19113.0,49.0,OK,39.9,OK
18285,19114.0,48.7,OK,39.9,OK


In [111]:
a.linearize()

,time,Viscosity,Viscosity_verbose,Temperature,Temperature_verbose
63,64.0,5.840351,image_sweep_check,0.003485,OK
64,65.0,5.841804,OK,0.003485,OK
65,66.0,5.841804,OK,0.003485,OK
66,67.0,5.841804,OK,0.003485,OK
67,68.0,5.843255,OK,0.003485,OK
...,...,...,...,...,...
18282,19111.0,3.889777,OK,0.003194,OK
18283,19112.0,3.897924,OK,0.003194,OK
18284,19113.0,3.891820,OK,0.003194,OK
18285,19114.0,3.885679,OK,0.003194,OK


In [112]:
base = a.copy()

# Plots

In [26]:
## Temporal plots
df =a.data
# df = test_data

fig, ax_v = plt.subplots()
ax_T = ax_v.twinx()

ax_v.scatter(df['time'], df['Viscosity'], color='red', marker='.')
ax_v.set_ylabel('Viscosity', color='red')
ax_v.set_xlabel('time')

ax_T.scatter(df['time'], df['Temperature'], color='blue', marker='.')
ax_T.set_ylabel('Temperature', color='blue')


Text(0, 0.5, 'Temperature')

In [25]:
## Temperature plots
df = a.data

fig, ax = plt.subplots()
colors= df['Viscosity_verbose'].replace({
    'OK':'g', 'OK_inner':'b', 'image_sweep_check':'r','combine_check':'w',
})
ax.scatter(x=df['Temperature'],y=df['Viscosity'],c= colors,s=5)

sns.lineplot(
    ax=ax,
    data=df,
    x="Temperature",
    y="Viscosity",
    estimator='mean',
    errorbar=("sd", 1),
    # err_style="bars", 
    label ='mean'  
)
sns.lineplot(
    ax=ax,
    data=df,
    x="Temperature",
    y="Viscosity",
    errorbar=('pi',68),
    estimator="median",
    label ='median'  
)

# df = b.data
# sns.lineplot(
#     ax=ax,
#     data=df,
#     x="Temperature",
#     y="Viscosity",
#     estimator='mean',
#     errorbar=("sd", 1),
#     # err_style="bars", 
#     label ='mean2'  
# )
# sns.lineplot(
#     ax=ax,
#     data=df,
#     x="Temperature",
#     y="Viscosity",
#     errorbar=('pi',68),
#     estimator="median",
#     label ='median2'  
# )

<Axes: xlabel='Temperature', ylabel='Viscosity'>

In [43]:
## fast OLS plot
def regress(data):
    reg = linear_model.LinearRegression(fit_intercept=True)
    X = np.array([data['Temperature']]).T
    Y = np.array(data['Viscosity'])
    reg.fit(X, Y)

    w_T = reg.coef_[0]
    w0_nu = reg.intercept_

    nu0 = np.exp(w0_nu)
    E = 8.314 * w_T /1000

    def TC_func(t,E=E,nu0 =nu0):
        Tt = 1 / (t+273.15)
        return nu0*np.exp(E/8.314*Tt)

    return nu0, E,TC_func

reg_data = a.data
nu0, EkJ, func = regress(reg_data)
sns.regplot(
    data=reg_data,
    x="Temperature",
    y="Viscosity",
    scatter=True,
    truncate=False,
    order=1,
    label=f'All:\nE= {EkJ:5.3f} kJ\nNu0= {nu0:5.3e} cP',
)

reg_data = a.data.groupby(by='Temperature')['Viscosity'].median().reset_index()
nu0, EkJ, func = regress(reg_data)
sns.regplot(
    data=reg_data,
    x="Temperature",
    y="Viscosity",
    scatter=False,
    truncate=False,
    order=1,
    label=f'Median:\nE= { EkJ:5.3f} kJ\nNu0= {nu0:5.3e} cP',
)

reg_data = a.data.groupby(by='Temperature')['Viscosity'].mean().reset_index()
nu0, EkJ, func = regress(reg_data)
sns.regplot(
    data=reg_data,
    x="Temperature",
    y="Viscosity",
    scatter=False,
    truncate=False,
    order=1,
    label=f'Mean:\nE= { EkJ:5.3f} kJ\nNu0= {nu0:5.3e} cP',
)
# reg_data = b.data
# nu0, EkJ, func = regress(reg_data)
# sns.regplot(
#     data=reg_data,
#     x="Temperature",
#     y="Viscosity",
#     scatter=True,
#     truncate=False,
#     order=1,
#     label=f'All2:\nE kJ= {EkJ:5.3f}\nNu0= {nu0:5.3e}',
# )

# reg_data = b.data.groupby(by='Temperature')['Viscosity'].median().reset_index()
# nu0, EkJ, func = regress(reg_data)
# sns.regplot(
#     data=reg_data,
#     x="Temperature",
#     y="Viscosity",
#     scatter=False,
#     truncate=False,
#     order=1,
#     label=f'Median2:\nE kJ= { EkJ:5.3f}\nNu0= {nu0:5.3e}',
# )

# reg_data = pd.concat([a.data, b.data])
# nu0, EkJ, func = regress(reg_data)
# sns.regplot(
#     data=reg_data,
#     x="Temperature",
#     y="Viscosity",
#     scatter=False,
#     truncate=False,
#     order=1,
#     label=f'All:\nE kJ= {EkJ:5.3f}\nNu0= {nu0:5.3e}',
# )

plt.legend()

# Regression

In [113]:
results = pd.DataFrame(columns=['E_kJ','dE_kJ','nu0_cP','dnu0_cP','name','w','desc'])

In [114]:
## Regression
reg_a = a.copy()

df = reg_a.data
df.rename(inplace=True, columns={'Temperature': 'E', 'Viscosity': 'nu'})
df['nu0'] = 1
result_OLS = sm.OLS(df['nu'], df[['E', 'nu0']]).fit()
means = result_OLS.params
conf_int = result_OLS.conf_int(0.005).loc

conf_int['nu0'] = np.exp(conf_int['nu0']) * 1e6
nu0_6 = np.exp(means['nu0']) * 1e6
nu0_6_diap = (conf_int['nu0'].max() - conf_int['nu0'].min()) / 2

conf_int['E'] = 8.314 * conf_int['E'] / 1000
EkJ = 8.314 * means['E'] / 1000
EkJ_diap = (conf_int['E'].max() - conf_int['E'].min()) / 2

exp_name = reg_a.experiment_name
w= reg_a.folder.split('(')[1][:-1]
print(
    f'Constants {exp_name} ({w}% mass):',
    f'E   = {EkJ: <6.3f} ± {EkJ_diap: <3.2f} kJ',
    f'Nu0 = {nu0_6: <6.3f} ± {nu0_6_diap: <3.2f} cP * 1e-6',
    sep='\n',
)
print(result_OLS.summary())

Constants Exp0_up (0% mass):
E   = 57.172 ± 0.10 kJ
Nu0 = 0.012  ± 0.00 cP * 1e-6
                            OLS Regression Results                            
Dep. Variable:                     nu   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 2.345e+06
Date:                Sun, 12 Nov 2023   Prob (F-statistic):               0.00
Time:                        19:46:05   Log-Likelihood:                 30669.
No. Observations:               17827   AIC:                        -6.133e+04
Df Residuals:                   17825   BIC:                        -6.132e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------

In [115]:
## Save result
desc= input('Description')
results.loc[len(results)] = [
    EkJ,
    EkJ_diap,
    nu0_6,
    nu0_6_diap,
    reg_a.experiment_name,
    w,
    desc
]
reg_a.dump(**results.loc[len(results)-1].to_dict())
results

,E_kJ,dE_kJ,nu0_cP,dnu0_cP,name,w,desc
0,57.17172,0.104819,0.012466,0.000531,Exp0_up,0,4


# Interactive

In [ ]:
#Time
def temp_plot(
    data,
    name,
    y2_axix=False,
    symbol='circle',
    colors=plotly.colors.qualitative.Plotly,
):
    verboses = data[f'{name}_verbose'].unique()
    rules = zip(
        verboses,
        [round(i) for i in np.linspace(0, len(colors) - 1, len(verboses))],
    )
    colors_rule = {key: colors[color] for key, color in rules}

    scatter = go.Scatter(
        x=data['time'],
        y=data[name],
        name=name,
        mode='markers',
        yaxis='y2' if y2_axix else 'y',
        text=data[f'{name}_verbose'],
        hovertemplate='%{y}<br>%{text}',
        marker=dict(
            size=5,
            symbol=symbol,
            color=data[f'{name}_verbose'].replace(colors_rule),
        ),
    )
    return scatter


a = a
fig = go.Figure()
fig.add_traces([
    temp_plot(
        a.data,
        'Viscosity',
        colors=plotly.colors.sequential.Aggrnyl,
    ),
    temp_plot(
        a.data,
        'Temperature',
        y2_axix=True,
        symbol='star',
        colors=plotly.colors.sequential.Sunsetdark_r,
    )
])
fig.update_layout(
    hovermode="x unified",
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    yaxis2=dict(
        overlaying='y',
        side='right',  # range=[15, 20],
    ),  # xaxis=dict(range=[0, 500], ),
)
fig.show()

In [ ]:
fig = go.Figure()
fig = px.scatter(
    a.data,
    x="Temperature",
    y="Viscosity",
    color='Viscosity_verbose',
)

## Mean and median
fig.add_traces([
    go.Scatter(name='Mean',
               x=a.groupT.mean().index,
               y=a.groupT.mean()['Viscosity'],
               mode='lines',
               line=dict(color='red')),
    go.Scatter(name='Median',
               x=a.groupT.median().index,
               y=a.groupT.median()['Viscosity'],
               mode='lines',
               line=dict(color='black')),
])

## Comparation
# test_plot= test
# fig.add_trace(
#     go.Scatter(name='Test',
#                x=test_plot.data['Temperature'],
#                y=test_plot.data['Viscosity'],
#                mode='markers',
#                line=dict(color='green'),
#                marker=dict(size=10)), )

# fig.add_traces([
#     go.Scatter(name='Mean comp',
#                x=test_plot.groupT.mean().index,
#                y=test_plot.groupT.mean()['Viscosity'],
#                mode='lines',
#                line=dict(color='red', width=2, dash='dot')),
#     go.Scatter(name='Median comp',
#                 x=test_plot.groupT.median().index,
#                 y=test_plot.groupT.median()['Viscosity'],
#                 mode='lines',
#                 line=dict(color='black', width=2, dash='dot')),
# ])

##Regression
# x = reg_data.index
# fig.add_trace(
#     go.Scatter(
#         name='Regression',
#         x=x,
#         y=reg_func(x),
#         mode='lines',
#         line=dict(color='green', width=2, dash='dot'),
#     ), )

fig.update_layout(hovermode="x unified", margin=dict(l=0, r=0, b=0, t=0, pad=0))
fig.show()